In [9]:
import requests
import pandas as pd
from datetime import datetime

# URL da API Prometheus
url = 'http://192.168.242.131:9090/api/v1/query?query=container_cpu_usage_seconds_total{pod=~".*postgres.*"}[10m]'

# Fazendo a requisição GET
response = requests.get(url)
data = response.json()  # Converte a resposta para dict

# Verifica se há resultados e estrutura esperada
results = data.get('data', {}).get('result', [])

# Lista para armazenar os registros
records = []

# Processa cada resultado
for item in results:
    # Extrai a série temporal (samples)
    for value in item.get('values', []):
        ts = float(value[0])  # Timestamp UNIX
        cpu_val = float(value[1])
        # Converte timestamp para Data/Hora legível
        dt = datetime.utcfromtimestamp(ts)
        
        # Extrai o nome do pod dos labels
        pod_name = item['metric'].get('pod', 'N/A')
        records.append({"timestamp": dt, "pod": pod_name, "cpu": cpu_val})

# Cria DataFrame
df = pd.DataFrame(records)

# Mostra resultado
df

,timestamp,pod,cpu
0,2025-09-04 16:16:28.724,postgres-deployment-76d6f4778d-ccs8g,45104.990671
1,2025-09-04 16:16:54.260,postgres-deployment-76d6f4778d-ccs8g,45104.990854
2,2025-09-04 16:17:11.110,postgres-deployment-76d6f4778d-ccs8g,45104.994082
3,2025-09-04 16:17:37.867,postgres-deployment-76d6f4778d-ccs8g,45104.994262
4,2025-09-04 16:18:07.502,postgres-deployment-76d6f4778d-ccs8g,45104.998204
...,...,...,...
94,2025-09-04 16:25:11.679,postgres-deployment-76d6f4778d-ccs8g,0.023826
95,2025-09-04 16:25:25.591,postgres-deployment-76d6f4778d-ccs8g,0.023826
96,2025-09-04 16:25:47.612,postgres-deployment-76d6f4778d-ccs8g,0.023826
97,2025-09-04 16:25:58.796,postgres-deployment-76d6f4778d-ccs8g,0.023826
